1. Implement weighted and unweighted F1, precision, recall for multi-class classification
2. Answer this question: could the weighted f1 score be outside the range of weighted precision and weighted recall?

In [18]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [285]:
X, y = make_classification(n_classes=3, 
                           weights=[0.5, 0.25, 0.01],
                           n_samples=10000, n_features=10, 
                           n_clusters_per_class=2, 
                    n_informative=10, n_redundant=0, n_repeated=0)

In [286]:
pd.DataFrame(y)[0].value_counts()

0    5778
1    3292
2     930
Name: 0, dtype: int64

In [287]:
lr = LogisticRegression(C=0.0001)
lr.fit(X, y)
y_pred = lr.predict(X)

In [288]:
def float_equal(n1, n2):
    return abs(n1-n2)<1e-8

In [289]:
accu_molly = sum((y == y_pred).astype(int))/len(y)
accu = accuracy_score(y_true=y, y_pred=y_pred)
print(float_equal(accu_molly,accu))
print(accu)

True
0.7676


In [314]:
def multi_class_weighting(y_true, y_pred, func):
    scores = []
    counts = []
    for each_label in set(y_true):
        y_true_ovr = (y_true==each_label).astype(int)
        y_pred_ovr = (y_pred==each_label).astype(int)
        counts.append(sum(y_true_ovr))
        scores.append(func(y_pred_ovr, y_true_ovr))
    print(scores)
    print(counts)
    weighted_score = sum(np.array(scores) * np.array(counts)/sum(counts))
    r_weighted_score = sum(np.array(scores) * (1/np.array(counts))/sum(1/np.array(counts)))
    return weighted_score, r_weighted_score
        
def precision(y_pred, y):
    positives = np.where(y_pred==1)[0]
    return sum(y[positives])/len(positives)

In [315]:
precision_molly, precision_molly_ = multi_class_weighting(y, y_pred, precision)
precision_sklearn = precision_score(y_true=y, y_pred=y_pred, average="weighted")
print(float_equal(precision_molly,precision_sklearn))
print(precision_sklearn)
print(precision_molly_)

[0.7930388637427813, 0.7401356350184957, 0.5558739255014327]
[5778, 3292, 930]
True
0.7535667815903011
0.6183817599535814


In [316]:
def recall(y_pred, y):
    positives = np.where(y==1)[0]
    return sum(y_pred[positives])/len(positives)

In [319]:
recall_molly, recall_molly_ = multi_class_weighting(y, y_pred, recall)
recall_sklearn = recall_score(y_true=y, y_pred=y_pred, average="weighted")
print(float_equal(recall_molly, recall_sklearn))
print(recall_sklearn)
print(recall_molly_)

[0.8793700242298373, 0.7293438639125152, 0.2086021505376344]
[5778, 3292, 930]
True
0.7676
0.3853130241415258


In [320]:
def f1(y_pred, y):
    p = precision(y_pred, y)
    r = recall(y_pred, y)
    return 2*p*r/(p+r)

In [321]:
f1_molly, f1_molly_ = multi_class_weighting(y, y_pred, f1)
f1_sklearn = f1_score(y_true=y, y_pred=y_pred, average="weighted")
print(float_equal(f1_molly, f1_sklearn))
print(f1_sklearn)
print(f1_molly_)

[0.8339762002462043, 0.7347001223990207, 0.3033620015637216]
[5778, 3292, 930]
True
0.7519473949414406
0.4469473826238354
